# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
import pathlib
import shutil


# Downsampling performance plateau for CXCL9

## Load original data

In [ ]:
epi_data_type = 'no_harm'
imm_data_type = 'imp_source(imm)_method(knn)_params(5)'

selection_method = 'f_regression' # 'f_regression' 'spearman'
n_feats = 100

imm = 'CXCL9'

path_load = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/{imm_data_type}/{epi_data_type}/{selection_method}_{n_feats}/{imm}"
path_save = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/revision/downsampling_performance_plateau_CXCL9"
data = pd.read_excel(f"{path_load}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path_load}/feats_con.xlsx", index_col=0)

## Generate downsampled datasets (using stratified splitting)

In [ ]:
n_splits = 5
n_repeats = 1
random_state = 1337

stratify_cat_parts = {
    'ctrl_central': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Central')].values,
    'ctrl_yakutia': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Yakutia')].values,
    'esrd': data.index[(data['Status'] == 'ESRD')].values,
}

for part, ids_part in stratify_cat_parts.items():
    
    quantiles = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    quantiles_part = pd.qcut(data.loc[ids_part, 'Age'].values, quantiles, labels=False, duplicates='drop')
    unique_part, counts_part = np.unique(quantiles_part, return_counts=True)
    
    k_fold_part = RepeatedStratifiedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=random_state
    )
    splits_part = k_fold_part.split(X=ids_part, y=quantiles_part, groups=quantiles_part)
    
    for split_id, (ids_trn_val, ids_tst) in enumerate(splits_part):
        data.loc[ids_part[ids_tst], f"Subsampling Split"] = split_id

for split_id in range(4):
    pathlib.Path(f"{path_save}/subsampling_{split_id}").mkdir(parents=True, exist_ok=True)
    data.loc[data[f"Subsampling Split"] <= split_id, :].to_excel(f"{path_save}/subsampling_{split_id}/data.xlsx")
    feats.to_excel(f"{path_save}/subsampling_{split_id}/feats.xlsx")

## Train downsampled data

In [ ]:
imm = 'CXCL9'

subsampling_split = 3

path = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/revision/downsampling_performance_plateau_CXCL9/subsampling_{subsampling_split}"
path_configs = "E:/Git/bbs/notebooks/immunology/003_EpImAge/immuno_regression_configs"
path_models = f"E:/Git/bbs/notebooks/immunology/003_EpImAge/pt/subsampling_{subsampling_split}"

tst_n_splits = 5
tst_n_repeats = 5
tst_random_state = 1337

val_n_splits = 4
val_n_repeats = 2
val_random_state = 1337

data = pd.read_excel(f"{path}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0).index.values.tolist()

stratify_cat_parts_all = {
    'ctrl_central': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Central')].values,
    'ctrl_yakutia': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Yakutia')].values,
    'esrd': data.index[(data['Status'] == 'ESRD')].values,
}


# Generate stratification
quantiles = [0.0, 0.25, 0.5, 0.75, 1.0]

for part_all, ids_all in stratify_cat_parts_all.items():
    trgt_all = data.loc[ids_all, 'Age'].values
    quantiles_all = pd.qcut(trgt_all, quantiles, labels=False, duplicates='drop')
    unique_all, counts_all = np.unique(quantiles_all, return_counts=True)
    k_fold_all = RepeatedStratifiedKFold(
        n_splits=tst_n_splits,
        n_repeats=tst_n_repeats,
        random_state=tst_random_state
    )
    splits_all = k_fold_all.split(X=ids_all, y=quantiles_all, groups=quantiles_all)
    
    for split_id, (ids_trn_val, ids_tst) in enumerate(splits_all):
        data.loc[ids_all[ids_trn_val], f"Split_{split_id}"] = "trn_val"
        data.loc[ids_all[ids_tst], f"Split_{split_id}"] = "tst"

samples = {}
for split_id in range(tst_n_splits * tst_n_repeats):
    samples[split_id] = {
        'test': data.index[data[f"Split_{split_id}"] == "tst"].values,
        'train_validation': data.index[data[f"Split_{split_id}"] == "trn_val"].values,
        'trains': {},
        'validations': {},
    }

    stratify_cat_parts_trnval = {
        'ctrl_central': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Central') & (data[f"Split_{split_id}"] == 'trn_val')].values,
        'ctrl_yakutia': data.index[(data['Status'] == 'Control') & (data['Region'] == 'Yakutia') & (data[f"Split_{split_id}"] == 'trn_val')].values,
        'esrd': data.index[(data['Status'] == 'ESRD') & (data[f"Split_{split_id}"] == 'trn_val')].values,
    }

    for part_trnval, ids_trnval in stratify_cat_parts_trnval.items():
        trgt_trnval = data.loc[ids_trnval, 'Age'].values
        quantiles_trnval = pd.qcut(trgt_trnval, quantiles, labels=False, duplicates='drop')
        unique_trnval, counts_trnval = np.unique(quantiles_trnval, return_counts=True)
        k_fold_trnval = RepeatedStratifiedKFold(
            n_splits=val_n_splits,
            n_repeats=val_n_repeats,
            random_state=val_random_state
        )
        splits_trnval = k_fold_trnval.split(X=ids_trnval, y=quantiles_trnval, groups=quantiles_trnval)
        
        for fold_id, (ids_trn, ids_val) in enumerate(splits_trnval):
            data.loc[ids_trnval[ids_trn], f"Split_{split_id}_Fold_{fold_id}"] = "trn"
            data.loc[ids_trnval[ids_val], f"Split_{split_id}_Fold_{fold_id}"] = "val"
         
    for fold_id in range(val_n_splits * val_n_repeats):
        samples[split_id]['trains'][fold_id] = data.index[data[f"Split_{split_id}_Fold_{fold_id}"] == "trn"].values
        samples[split_id]['validations'][fold_id] = data.index[data[f"Split_{split_id}_Fold_{fold_id}"] == "val"].values

    samples[split_id]['cv_indexes'] = [
        (
            np.where(data.index[data[f"Split_{split_id}"] == "trn_val"].isin(data.index[(data[f"Split_{split_id}"] == "trn_val") & (data[f"Split_{split_id}_Fold_{i}"] == 'trn')]))[0],
            np.where(data.index[data[f"Split_{split_id}"] == "trn_val"].isin(data.index[(data[f"Split_{split_id}"] == "trn_val") & (data[f"Split_{split_id}_Fold_{i}"] == 'val')]))[0],
        )
        for i in range(val_n_splits * val_n_repeats)
    ]

for split_id in range(tst_n_splits * tst_n_repeats):
    for fold_id in range(val_n_splits * val_n_repeats):
        test_samples = samples[split_id]['test']
        train_samples = samples[split_id]['trains'][fold_id]
        validation_samples = samples[split_id]['validations'][fold_id]

        intxns = {
            'train_validation': set.intersection(set(train_samples), set(validation_samples)),
            'validation_test': set.intersection(set(validation_samples), set(test_samples)),
            'train_test': set.intersection(set(train_samples), set(test_samples))
        }
        
        for intxn_name, intxn_samples in intxns.items():
            if len(intxn_samples) > 0:
                print(f"Non-zero {intxn_name} intersection ({len(intxn_samples)}) for {split_id} Split and {fold_id} Fold!")

with open(f"{path}/samples_tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats}).pickle", 'wb') as handle:
    pickle.dump(samples, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Load non-model configs
data_config = read_parse_config(f"{path_configs}/DataConfig.yaml", DataConfig)
data_config['target'] = [f"{imm}_log"]
data_config['continuous_cols'] = feats
trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
pathlib.Path(path_models).mkdir(parents=True, exist_ok=True)
trainer_config['checkpoints_path'] = path_models
optimizer_config = read_parse_config(f"{path_configs}/OptimizerConfig.yaml", OptimizerConfig)

lr_find_min_lr = 1e-8
lr_find_max_lr = 1
lr_find_num_training = 128
lr_find_mode = "exponential"
lr_find_early_stop_threshold = 4.0


# GANDALF Search Space
search_space = {
    "model_config__gflu_stages": [6],
    "model_config__gflu_dropout": [0.1],
    "model_config__gflu_feature_init_sparsity": [0.3],
    "model_config.head_config__dropout": [0.1],
    "model_config__learning_rate": [0.001],
    "model_config__seed": [1899],
}
grid_size = np.prod([len(p_vals) for _, p_vals in search_space.items()])
print(grid_size)

head_config = LinearHeadConfig(
    layers="",
    activation='ReLU',
    dropout=0.1,
    use_batch_norm=False,
    initialization="kaiming"
).__dict__

model_list = []
for i, params in enumerate(ParameterGrid(search_space)):
    head_config_tmp = copy.deepcopy(head_config)
    head_config_tmp['dropout'] = params['model_config.head_config__dropout']
    model_config = read_parse_config(f"{path_configs}/models/GANDALFConfig.yaml", GANDALFConfig)
    model_config['gflu_stages'] = params['model_config__gflu_stages']
    model_config['gflu_feature_init_sparsity'] = params['model_config__gflu_feature_init_sparsity']
    model_config['gflu_dropout'] = params['model_config__gflu_dropout']
    model_config['learning_rate'] = params['model_config__learning_rate']
    model_config['seed'] = params['model_config__seed']
    model_config['head_config'] = head_config_tmp
    model_list.append(GANDALFConfig(**model_config))


# Model Sweep Training
common_params = {
    "task": "regression",
}

seed = 1337 # 1337, 55763, 40279, 87571, 234461

dfs_result = []
for split_id, split_dict in samples.items():
    for fold_id in split_dict['trains']:
        test = data.loc[split_dict['test'], feats + [f"{imm}_log"]]
        train = data.loc[split_dict['trains'][fold_id], feats + [f"{imm}_log"]]
        validation = data.loc[split_dict['validations'][fold_id], feats + [f"{imm}_log"]]

        trainer_config['seed'] = seed
        trainer_config['checkpoints'] = 'valid_loss'
        trainer_config['load_best'] = True
        trainer_config['auto_lr_find'] = True
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sweep_df, best_model = model_sweep_custom(
                task="regression",
                train=train,
                validation=validation,
                test=test,
                data_config=data_config,
                optimizer_config=optimizer_config,
                trainer_config=trainer_config,
                model_list=model_list,
                common_model_args=common_params,
                metrics=["mean_absolute_error", "pearson_corrcoef"],
                metrics_params=[{}, {}],
                metrics_prob_input=[False, False],
                rank_metric=("mean_absolute_error", "lower_is_better"),
                return_best_model=True,
                seed=seed,
                progress_bar=False,
                verbose=False,
                suppress_lightning_logger=True,
                min_lr = lr_find_min_lr,
                max_lr = lr_find_max_lr,
                num_training = lr_find_num_training,
                mode = lr_find_mode,
                early_stop_threshold = lr_find_early_stop_threshold,
            )
        sweep_df['seed'] = seed
        sweep_df['split_id'] = split_id
        sweep_df['fold_id'] = fold_id
        sweep_df["train_more"] = False
        sweep_df.loc[(sweep_df["train_loss"] > sweep_df["test_loss"]) | (sweep_df["train_loss"] > sweep_df["validation_loss"]), "train_more"] = True
        sweep_df["validation_test_mean_loss"] = (sweep_df["validation_loss"] + sweep_df["test_loss"]) / 2.0
        sweep_df["train_validation_test_mean_loss"] = (sweep_df["train_loss"] + sweep_df["validation_loss"] + sweep_df["test_loss"]) / 3.0
        
        dfs_result.append(sweep_df)
        
        fn_suffix = (f"models({len(model_list)})_"
                     f"tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_"
                     f"val({val_random_state}_{val_n_splits}_{val_n_repeats})")
        try:
            df_result = pd.concat(dfs_result, ignore_index=True)
            df_result.style.background_gradient(cmap="RdYlGn_r").to_excel(f"{trainer_config['checkpoints_path']}/{fn_suffix}.xlsx")
        except PermissionError:
            pass

## Save best models

In [ ]:
n_models = 1

explain_method = "GradientShap"
explain_baselines = "b|1000"
explain_n_feats_to_plot = 25

fn_sweep = (
    f"models({n_models})_"
    f"tst({tst_random_state}_{tst_n_splits}_{tst_n_repeats})_val({val_random_state}_{val_n_splits}_{val_n_repeats})"
)

df_sweeps = pd.read_excel(f"{trainer_config['checkpoints_path']}/{fn_sweep}.xlsx", index_col=0)
path_to_candidates = f"{trainer_config['checkpoints_path']}/candidates/{fn_sweep}"
pathlib.Path(path_to_candidates).mkdir(parents=True, exist_ok=True)
df_sweeps.style.background_gradient(cmap="RdYlGn_r").to_excel(f"{path_to_candidates}/sweep.xlsx")

models_ids = [
97
]

df_sweeps.loc[models_ids, :].style.background_gradient(cmap="RdYlGn_r").to_excel(f"{path_to_candidates}/selected.xlsx")

for model_id in models_ids:
    
    split_id = df_sweeps.at[model_id, 'split_id']
    fold_id = df_sweeps.at[model_id, 'fold_id']
    split_dict = samples[split_id]
    
    test = data.loc[split_dict['test'], feats + [f"{imm}_log"]]
    train = data.loc[split_dict['trains'][fold_id], feats + [f"{imm}_log"]]
    validation = data.loc[split_dict['validations'][fold_id], feats + [f"{imm}_log"]]
    
    model_dir = str(pathlib.Path(df_sweeps.at[model_id, 'checkpoint']).parent).replace('\\', '/') + '/' + pathlib.Path(df_sweeps.at[model_id, 'checkpoint']).stem
    model = TabularModel.load_model(model_dir)
    pathlib.Path(f"{path_to_candidates}/{model_id}").mkdir(parents=True, exist_ok=True)
    shutil.copytree(model_dir, f"{path_to_candidates}/{model_id}", dirs_exist_ok=True)
    
    df = data.loc[:, data_config['target']]
    df.loc[train.index, 'Group'] = 'Train'
    df.loc[validation.index, 'Group'] = 'Validation'
    df.loc[test.index, 'Group'] = 'Test'
    df['Prediction'] = model.predict(data)
    df['Error'] = df['Prediction'] - df[data_config['target'][0]]
    df.to_excel(f"{path_to_candidates}/{model_id}/df.xlsx")
    
    colors_groups = {
        'Train': 'chartreuse',
        'Validation': 'dodgerblue',
        'Test': 'crimson',
    }
    
    df_metrics = pd.DataFrame(
        index=list(colors_groups.keys()),
        columns=['mean_absolute_error', 'pearson_corrcoef', 'bias']
    )
    for group in colors_groups.keys():
        pred = torch.from_numpy(df.loc[df['Group'] == group, 'Prediction'].values)
        real = torch.from_numpy(df.loc[df['Group'] == group, data_config['target'][0]].values)
        df_metrics.at[group, 'mean_absolute_error'] = mean_absolute_error(pred, real).numpy()
        df_metrics.at[group, 'pearson_corrcoef'] = pearson_corrcoef(pred, real).numpy()
        df_metrics.at[group, 'bias'] = np.mean(df.loc[df['Group'] == group, 'Error'].values)
    df_metrics.to_excel(f"{path_to_candidates}/{model_id}/metrics.xlsx", index_label="Metrics")
    
    sns.set_theme(style='whitegrid')
    xy_min = df[[data_config['target'][0], 'Prediction']].min().min()
    xy_max = df[[data_config['target'][0], 'Prediction']].max().max()
    xy_ptp = xy_max - xy_min
    fig, ax = plt.subplots(figsize=(4.5, 4))
    scatter = sns.scatterplot(
        data=df,
        x=data_config['target'][0],
        y="Prediction",
        hue="Group",
        palette=colors_groups,
        linewidth=0.2,
        alpha=0.75,
        edgecolor="k",
        s=20,
        hue_order=list(colors_groups.keys()),
        ax=ax
    )
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=ax
    )
    ax.set_title(f"{df_sweeps.at[model_id, 'model']} ({df_sweeps.at[model_id, '# Params']} params, {df_sweeps.at[model_id, 'epochs']} epochs)")
    ax.set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    ax.set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    plt.gca().set_aspect('equal', adjustable='box')
    fig.savefig(f"{path_to_candidates}/{model_id}/scatter.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/scatter.pdf", bbox_inches='tight')
    plt.close(fig)
    
    df_fig = df.loc[:, ['Error', 'Group']]
    groups_rename = {
        group: f"{group}" + "\n" +
               fr"MAE: {df_metrics.at[group, 'mean_absolute_error']:0.2f}" + "\n"
               fr"Pearson $\rho$: {df_metrics.at[group, 'pearson_corrcoef']:0.2f}" + "\n" +
               fr"$\langle$Error$\rangle$: {df_metrics.at[group, 'bias']:0.2f}" 
        for group in colors_groups
    }
    colors_groups_violin = {groups_rename[group]: colors_groups[group] for group in colors_groups}
    df_fig['Group'].replace(groups_rename, inplace=True)
    sns.set_theme(style='whitegrid')
    fig, ax = plt.subplots(figsize=(7, 4))
    violin = sns.violinplot(
        data=df_fig,
        x='Group',
        y='Error',
        palette=colors_groups_violin,
        scale='width',
        order=list(colors_groups_violin.keys()),
        saturation=0.75,
        legend=False,
        ax=ax
    )
    ax.set_xlabel('')
    fig.savefig(f"{path_to_candidates}/{model_id}/violin.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path_to_candidates}/{model_id}/violin.pdf", bbox_inches='tight')
    plt.close(fig)
    
    try:
        explanation = model.explain(data, method=explain_method, baselines=explain_baselines)
        explanation.index = data.index
        explanation.to_excel(f"{path_to_candidates}/{model_id}/explanation.xlsx")
        
        sns.set_theme(style='whitegrid')
        fig = shap.summary_plot(
            shap_values=explanation.loc[:, feats].values,
            features=data.loc[:, feats].values,
            feature_names=feats,
            max_display=explain_n_feats_to_plot,
            plot_type="violin",
            show=False,
        )
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_beeswarm.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_beeswarm.pdf", bbox_inches='tight')
        plt.close(fig)
        
        sns.set_theme(style='whitegrid')
        fig = shap.summary_plot(
            shap_values=explanation.loc[:, feats].values,
            features=data.loc[:, feats].values,
            feature_names=feats,
            max_display=explain_n_feats_to_plot,
            plot_type="bar",
            show=False,
        )
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_bar.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_to_candidates}/{model_id}/explain_bar.pdf", bbox_inches='tight')
        plt.close(fig)
    
    except NotImplementedError:
        pass

## Plot test Pearson’s R from dataset size

In [ ]:
imm = 'CXCL9'

path = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/revision/downsampling_performance_plateau_CXCL9"

df = pd.read_excel(f"{path}/downsampling.xlsx")

fig, ax = plt.subplots(figsize=(5, 3.5), layout='constrained')
lineplot = sns.lineplot(
    data=df,
    linewidth=2,
    x='Subsampling Part, %',
    y="CXCL9 Pearson\'s R on Test",
    color='crimson',
    marker='o',
    markersize=10,
    markerfacecolor='crimson',
    markeredgecolor='black',
    ax=ax
)
lineplot.set(xlim=(15, 105))
lineplot.set(xticks=[20, 40, 60, 80, 100])
lineplot.set(ylim=(0.65, 0.85))
plt.savefig(f"{path}/lineplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/lineplot.pdf", bbox_inches='tight')
plt.close(fig)

# Supplementary figure for models' performance

In [ ]:
path = f"E:/YandexDisk/Work/bbd/immunology/003_EpImAge/revision/models_performance"

dfs_imms = pd.read_excel(f"{path}/SupplementaryTableS3.xlsx", sheet_name=None, skiprows=0)
dfs_imms.pop('EpInflammAge')
df_age = pd.read_excel(f"{path}/SupplementaryTableS3.xlsx", sheet_name='EpInflammAge', skiprows=1)

colors_models = {
    'Elastic Net': 'goldenrod',
    'MLP': 'orchid',
    'DANet': 'chartreuse',
    'FT-Transformer': 'deepskyblue',
    'GANDALF': 'crimson',
}

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 2.2), layout='constrained')
barplot = sns.barplot(
    df_age,
    y="Model",
    x="Pearson Rho Test",
    hue="Model",
    palette=colors_models,
    ax=ax
)
ax.set_title('EpInflammAge')
ax.set_ylabel('')
ax.set_xlabel('Pearson\'s R Test')
patch_w = [patch.get_width() for patch in barplot.patches]
idx_best = np.argmax(patch_w)
barplot.patches[idx_best].set_edgecolor('navy')
barplot.patches[idx_best].set_linewidth(2)
barplot.bar_label(barplot.containers[idx_best], labels=[f"Best: {df_age.at[idx_best, 'Pearson Rho Test']:0.3f}"], label_type='center', color='navy')
plt.savefig(f"{path}/barplot_age.pdf", bbox_inches='tight')
plt.savefig(f"{path}/barplot_age.png", bbox_inches='tight', dpi=200)
plt.close(fig)

n_cols = 4
n_rows = 8
sns.set_theme(style='ticks')
fig, axs = plt.subplots(
    nrows=n_rows,
    ncols=n_cols,
    figsize=(n_cols * 3.0, n_rows * 2.0),
    gridspec_kw={'wspace':0.10, 'hspace': 0.20}, 
    sharex=True,
    sharey=True,
    layout='constrained'
)
for feat_id, feat in enumerate(dfs_imms.keys()):
    row_id, col_id = divmod(feat_id, n_cols)
    
    barplot = sns.barplot(
        dfs_imms[feat],
        y="Model",
        x="Pearson Rho Test",
        hue="Model",
        palette=colors_models,
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set_title(feat)
    axs[row_id, col_id].set_ylabel('')
    axs[row_id, col_id].set_xlabel('Pearson\'s R Test')
    patch_w = [patch.get_width() for patch in barplot.patches]
    idx_best = np.argmax(patch_w)
    barplot.patches[idx_best].set_edgecolor('navy')
    barplot.patches[idx_best].set_linewidth(2)
    barplot.bar_label(barplot.containers[idx_best], labels=[f"Best: {dfs_imms[feat].at[idx_best, 'Pearson Rho Test']:0.3f}"], label_type='center', color='navy')

fig.savefig(f"{path}/barplot_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/barplot_feats.pdf", bbox_inches='tight')
plt.close(fig)